In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
# If running type is GPU, run this cell

!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]

def printm():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))


printm()


  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=53d496ab661082ccf843c37c6ef1a5fefcad3e108f9396a6d927c47b6e480bd3
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.7 GB  | Proc size: 161.3 MB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


In [ ]:
# If run type is TPU, run this cell 
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

try:
  # imports the torch_xla package
  import torch_xla
  import torch_xla.core.xla_model as xm

  device = xm.xla_device()

except:
  VERSION = "20200325"  #@param ["1.5" , "20200325", "nightly"]
  !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
  !python pytorch-xla-env-setup.py --version $VERSION

  # imports pytorch
  import torch

  # imports the torch_xla package
  import torch_xla
  import torch_xla.core.xla_model as xm

  device = xm.xla_device()

Python library

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import re
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize, sent_tokenize

from html.parser import HTMLParser
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler

import torch
from torch.utils.data import TensorDataset, DataLoader, random_split, RandomSampler, SequentialSampler

import time
import datetime

import warnings
warnings.filterwarnings('ignore')

try:
  import transformers as ppb
except:
  !pip install transformers
  import transformers as ppb

from transformers import AdamW, BertConfig, get_linear_schedule_with_warmup

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
     |████████████████████████████████| 757kB 6.4MB/s 
     |████████████████████████████████| 1.1MB 18.1MB/s 
     |████████████████████████████████| 890kB 38.1MB/s 
     |████████████████████████████████| 3.0MB 40.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=de541b37361687dc5c13233be5edb79ac9ad07317336048360ab8269195e44b9
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [5]:
# If running on GPU, run this cell to enable cuda
if torch.cuda.is_available():    
  
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [6]:
def text_cleaning(text):
    # converting HTML character codes to ASCII code
    parser = HTMLParser()
    text = parser.unescape(text)

    text = re.sub(r'<[^>]+>', '', text)  # removing HTML tags
    text = re.sub(r'(?:\#+[\w_]+[\w\'_\-]*[\w_]+)', '', text)  # removing hash-tags
    text = re.sub('\n', ' ', text)  # remove new line
    text = re.sub('@', '', text)  # remove @ sign
    #text = re.sub(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', '',text)  # removing URLs
    text = re.sub(r'(?:[\ufffd]+)', '', text)  # removing special characters
    text = text.lower()

    return text

In [7]:
def format_time(elapsed):
    # display the elapsed time when loading the data into the BERT model
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [8]:
def get_accuracy(pred, true):
    pred_flat = np.argmax(pred, axis=1).flatten()
    true_flat = true.flatten()
    return np.sum(pred_flat == true_flat) / len(true_flat)

Load data

In [9]:
df = pd.read_csv('/content/gdrive/My Drive/Amazon Seller Forum/final_merged_data.csv')
df.drop('Unnamed: 0',axis=1,inplace=True)

In [ ]:
print(df.isna().sum())
print('')

nan_row = np.where(df.isna())[0][0]
print(df.iloc[nan_row,:])

Title              0
Post Author        0
Leading Comment    1
Reply Comments     0
Category           0
Forum              0
dtype: int64

Title              Israel Cohen’s Success Story1
Post Author                          Trent-Admin
Leading Comment                              NaN
Reply Comments                                []
Category                             Misc Topics
Forum                                   Flowster
Name: 8832, dtype: object


In [10]:
df.dropna(axis=0,subset=['Leading Comment'],inplace = True)
df.reset_index(drop=True,inplace=True)

In [11]:
df['Category'].value_counts()

Selling on Amazon                                     2100
Account Health                                        1549
Fulfillment By Amazon                                 1500
Global Selling                                         600
Amazon Pay                                             600
Groups                                                 494
Site Feedback                                          300
Amazon Marketplace Web Service (MWS)                   300
US Announcements                                       300
Amazon Sponsored Products                              300
Amazon Custom                                          274
Login With Amazon                                      199
Health,Safety,Sustainability,Security & Compliance      63
Flowster-specific                                       59
Amazon Specific                                         53
Product Sourcing                                        53
Human Resources                                         

In [12]:
df['num_sent'] = df['Leading Comment'].apply(lambda x: len(sent_tokenize(x)))
df['num_word']= df['Leading Comment'].apply(lambda x: len(word_tokenize(x)))
df['num_capword_title'] = df['Title'].apply(lambda x: sum(word[0].isupper() for word in word_tokenize(x)))

row_drop_idx = df[df['Category'] == 'Store & Website Management'].index[0]

df.drop(df.index[row_drop_idx],inplace=True)
df.reset_index(drop=True,inplace=True)

row_drop_idx = df[df['Category'] == 'eCommerce Marketplaces'].index[0]
df.drop(df.index[row_drop_idx],inplace=True)
df.reset_index(drop=True,inplace=True)

In [13]:
comments = df['Leading Comment'].apply(lambda x:text_cleaning(x))
#df['corpus'] = df['Title']+" "+ df['Leading Comment']
#df['corpus'] = df['corpus'].apply(lambda x:text_cleaning(x))
#corpus = df['corpus']

#df_amazon = df[df['Forum']=='Amazon']
#comments = df_amazon['Leading Comment'].apply(lambda x:text_cleaning(x))

Load Transformer model

In [14]:
#from transformers import XLNetLMHeadModel, XLNetTokenizer

#tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")
#model = XLNetLMHeadModel.from_pretrained("xlnet-base-cased",
                                         #output_attentions = False, # Whether the model returns attentions weights.
                                         #output_hidden_states = False, # Whether the model returns all hidden-states.
                                         #)
#model_class, tokenizer_class, pretrained_weights = (ppb.BertForSequenceClassification, ppb.BertTokenizer, 'bert-base-cased')

# Load pretrained model/tokenizer

tokenizer = ppb.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

model = ppb.DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=24)
model.to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

tokenize leading comments

In [15]:
label_encoder = LabelEncoder()
inputs = []
attention_masks = []

# Tokenize all of the sentences and map the tokens to thier word IDs.
    #   1. Tokenize the sentence.
    #   2. Prepend the `[CLS]` token to the start.
    #   3. Append the `[SEP]` token to the end.
    #   4. Map tokens to their IDs.
    #   5. Pad or truncate the sentence to `max_length`
    #   6. Create attention masks for [PAD] tokens.

for comment in comments:
#for comment in corpus:

    encoded_dict = tokenizer.encode_plus(
                        comment,                      
                        add_special_tokens = True, 
                        truncation=True,
                        max_length = 512,           
                        pad_to_max_length = True,
                        return_overflowing_tokens=True,
                        stride=70,
                        return_attention_mask = True,  
                        return_tensors = 'pt',     
                   )
    
    # Add the encoded sentence to the list.    
    inputs.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors and load them to GPU/TPU
inputs = torch.cat(inputs, dim=0)#.to(device)
attention_masks = torch.cat(attention_masks, dim=0)#.to(device)

# Encoding the labels and convert them to tensor 
labels = label_encoder.fit_transform(df['Category'])
labels = torch.tensor(labels)

Prepare training dataset and validating dataset

In [16]:
batch_size = 16

train_size = int(len(comments)*0.7)
val_size = int(len(comments)*0.1)
test_size = len(comments) - train_size - val_size

dataset = TensorDataset(inputs,attention_masks,labels)

train_set, val_set, test_set = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_set, sampler = RandomSampler(train_set), batch_size = batch_size)

val_loader = DataLoader(val_set, sampler = SequentialSampler(val_set), batch_size = batch_size)

test_loader = DataLoader(test_set, sampler = SequentialSampler(test_set), batch_size = batch_size)

In [17]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, 
                  eps = 1e-8)

epochs = 3

# Total number of training steps = [number of batches]*[number of epochs]. 
total_steps = len(train_loader)*epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

Training Model 

In [19]:
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

training_log = []

for epoch in range(epochs):
  time_start = time.time()
  total_train_loss = 0

  print('begin training')
  print('')

  model.train()
  
  for step, batch in enumerate(train_loader):
    if step % 10 == 0 and not step == 0:
      elapsed = format_time(time.time()-time_start)
      print('Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_loader), elapsed))
    
    b_inputs = batch[0].to(device)
    b_attention_masks = batch[1].to(device)
    b_label = batch[2].to(device)

    model.zero_grad()
    loss, logit = model(b_inputs,attention_mask=b_attention_masks, labels = b_label)
    
    optimizer.zero_grad()
    total_train_loss += loss.item()
    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    optimizer.step()
    scheduler.step()
    

  avg_train_loss = total_train_loss/len(train_loader)
  training_time = format_time(time.time() - time_start)
  print('')
  print('Average training loss: {0:.2f}'.format(avg_train_loss))
  print('Training epcoh time: {:}'.format(training_time))
  
  print('')
  print('begin validation')

  time_start = time.time()

  model.eval()

  total_eval_acc = 0
  total_eval_loss = 0
  nb_eval_steps = 0

  for batch in val_loader:
    b_inputs = batch[0].cuda()
    b_attention_masks = batch[1].cuda()
    b_label = batch[2].cuda() 

    with torch.no_grad():
      (loss, logits) = model(b_inputs,attention_mask=b_attention_masks, labels = b_label)

    total_eval_loss += loss.item()

    logits = logits.detach().cpu().numpy()
    label = b_label.to('cpu').numpy()

    total_eval_acc += get_accuracy(logits, label)
  
  avg_val_accuracy = total_eval_acc/len(val_loader)
  print("Accuracy: {0:.2f}".format(avg_val_accuracy))

  avg_val_loss = total_eval_loss/len(val_loader)

  validation_time = format_time(time.time() - time_start)
  
  print("Validation Loss: {0:.2f}".format(avg_val_loss))
  print("Validation time: {:}".format(validation_time))

  training_log.append(
        {
            'epoch': epoch+1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )
  
print('training complete')

begin training

Batch    10  of    387.    Elapsed: 0:00:15.
Batch    20  of    387.    Elapsed: 0:00:30.
Batch    30  of    387.    Elapsed: 0:00:45.
Batch    40  of    387.    Elapsed: 0:01:00.
Batch    50  of    387.    Elapsed: 0:01:15.
Batch    60  of    387.    Elapsed: 0:01:29.
Batch    70  of    387.    Elapsed: 0:01:44.
Batch    80  of    387.    Elapsed: 0:01:59.
Batch    90  of    387.    Elapsed: 0:02:14.
Batch   100  of    387.    Elapsed: 0:02:29.
Batch   110  of    387.    Elapsed: 0:02:44.
Batch   120  of    387.    Elapsed: 0:02:58.
Batch   130  of    387.    Elapsed: 0:03:13.
Batch   140  of    387.    Elapsed: 0:03:28.
Batch   150  of    387.    Elapsed: 0:03:43.
Batch   160  of    387.    Elapsed: 0:03:58.
Batch   170  of    387.    Elapsed: 0:04:13.
Batch   180  of    387.    Elapsed: 0:04:28.
Batch   190  of    387.    Elapsed: 0:04:42.
Batch   200  of    387.    Elapsed: 0:04:57.
Batch   210  of    387.    Elapsed: 0:05:12.
Batch   220  of    387.    Elapsed: 0:0

Evaluating model

In [22]:
# switch model to evaluation mode
model.eval()

# Tracking variables 
pred_labels = []
true_labels = []

for batch in test_loader:

    b_inputs = batch[0].to(device)
    b_attention_masks = batch[1].to(device)
    b_label = batch[2].to(device) 
    
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_inputs,
                      attention_mask=b_attention_masks)

    logits = outputs[0]

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    labels = b_label.to('cpu').numpy()
    
    # Store predictions and true labels
    pred_labels.append(logits)
    true_labels.append(labels)

In [35]:
flat_pred_labels = [item for sublist in pred_labels for item in sublist]
flat_pred_labels = np.argmax(flat_pred_labels, axis=1).flatten()

# Combine the correct labels for each batch into a single list.
flat_true_labels = [item for sublist in true_labels for item in sublist]

print(classification_report(flat_true_labels, flat_pred_labels))

              precision    recall  f1-score   support

           0       0.81      0.88      0.84       293
           1       0.85      0.40      0.54        58
           2       0.62      0.68      0.65        65
           3       0.65      0.69      0.67       118
           4       0.00      0.00      0.00        13
           5       0.64      0.79      0.71        61
           6       0.00      0.00      0.00         3
           7       0.00      0.00      0.00        12
           8       0.00      0.00      0.00         1
           9       0.70      0.74      0.72       313
          10       0.55      0.47      0.50       120
          11       0.71      0.64      0.67        88
          12       0.00      0.00      0.00        12
          13       0.00      0.00      0.00         3
          14       0.69      0.73      0.71        37
          15       0.00      0.00      0.00         2
          17       0.00      0.00      0.00        11
          18       0.56    

Saving model

In [31]:
import os

output_dir = '/content/gdrive/My Drive/Amazon Seller Forum/bert_model/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Saving model to /content/gdrive/My Drive/Amazon Seller Forum/bert_model/


('/content/gdrive/My Drive/Amazon Seller Forum/bert_model/vocab.txt',
 '/content/gdrive/My Drive/Amazon Seller Forum/bert_model/special_tokens_map.json',
 '/content/gdrive/My Drive/Amazon Seller Forum/bert_model/added_tokens.json')

loading model for futre

In [37]:
# Load a trained model and vocabulary that you have fine-tuned
#model = ppb.BertForSequenceClassification.from_pretrained(output_dir)
#tokenizer = ppb.DistilBertTokenizer.from_pretrained(output_dir)

# Copy the model to the GPU.
#model.to(device)

Some weights of the model checkpoint at /content/gdrive/My Drive/Amazon Seller Forum/bert_model/ were not used when initializing BertForSequenceClassification: ['distilbert.embeddings.word_embeddings.weight', 'distilbert.embeddings.position_embeddings.weight', 'distilbert.embeddings.LayerNorm.weight', 'distilbert.embeddings.LayerNorm.bias', 'distilbert.transformer.layer.0.attention.q_lin.weight', 'distilbert.transformer.layer.0.attention.q_lin.bias', 'distilbert.transformer.layer.0.attention.k_lin.weight', 'distilbert.transformer.layer.0.attention.k_lin.bias', 'distilbert.transformer.layer.0.attention.v_lin.weight', 'distilbert.transformer.layer.0.attention.v_lin.bias', 'distilbert.transformer.layer.0.attention.out_lin.weight', 'distilbert.transformer.layer.0.attention.out_lin.bias', 'distilbert.transformer.layer.0.sa_layer_norm.weight', 'distilbert.transformer.layer.0.sa_layer_norm.bias', 'distilbert.transformer.layer.0.ffn.lin1.weight', 'distilbert.transformer.layer.0.ffn.lin1.bias',